In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import torch
from transformers import GPTNeoForSequenceClassification, GPTNeoForCausalLM, GPT2Tokenizer
from model import GPTNeoForSequenceClassificationBinary
import lightning.pytorch as pl
from model import GPTSequenceClassifiationModule

In [139]:


test_name = 'data/wordnet_all_false_july4_test.csv' #'data/ccs_test.csv'
actual_tree = 'data/bansal_wordnet_true_pairs.csv' #'data/acm_ccs_clean.csv'
model_path = '/root/autodl-tmp/runs/lora_8.ckpt'
save_path = 'results/wordnet/lora.csv'

model = GPTSequenceClassifiationModule.load_from_checkpoint(model_path)
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-1.3B and are newly initialized: ['score.weight', 'transformer.h.13.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.19.attn.attention.bias', 'transformer.h.3.attn.attention.bias', 'transformer.h.1.attn.attention.bias', 'transformer.h.21.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.5.attn.attention.bias', 'transformer.h.11.attn.attention.bias', 'transformer.h.17.attn.attention.bias', 'transformer.h.15.attn.attention.bias', 'transformer.h.23.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1576962 || all params: 1317152770 || trainable%: 0.11972506423837229
PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPTNeoForSequenceClassification(
      (transformer): GPTNeoModel(
        (wte): Embedding(50257, 2048)
        (wpe): Embedding(2048, 2048)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPTNeoBlock(
            (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (attn): GPTNeoAttention(
              (attention): GPTNeoSelfAttention(
                (attn_dropout): Dropout(p=0.0, inplace=False)
                (resid_dropout): Dropout(p=0.0, inplace=False)
                (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
                (v_proj): Linear(
                  in_features=2048, out_features=2048, bias=False
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
              

In [140]:

model=model.eval()
model=model.cuda()

In [141]:
# test_name='/root/Taxonomy-GPT/data/wordnet_all_false_july4_test.csv'
# actual_tree='/root/Taxonomy-GPT/data/bansal_wordnet_true_pairs.csv'
# test_name='/root/Taxonomy-GPT/data/ccs_test.csv'
# actual_tree='/root/Taxonomy-GPT/data/acm_ccs_clean.csv'

In [142]:
from dataset import get_taxonomy_dataset, get_taxonomy_dataset_binary
dataset = get_taxonomy_dataset_binary(test_name, entire_dataset=True,remove_columns=True)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-efeade2dc67e1dd3.arrow


In [143]:
dataset

{'test': Dataset({
     features: ['flag', 'input_ids', 'labels'],
     num_rows: 53968
 })}

In [144]:
def format_dataset(example,tokenizer):
    prompt_template = "I am doing the taxonomy research. I think {child} is a subtopic of {parent}"
    prompt = prompt_template.format(child=example['child'], parent=example['parent'])
    example['input_ids'] = tokenizer.encode(prompt)
    return example

def test_dataloader(str_path,batch_size):
    dataset= get_taxonomy_dataset_binary(str_path,entire_dataset=True)
    dataset = dataset['test']
    dataset = dataset.map(lambda example: format_dataset(example,tokenizer))
    dataset = dataset.remove_columns(['child', 'parent', 'group'])

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=DataCollatorWithPadding(tokenizer),
        num_workers=4,
    )

In [145]:
from torch.utils.data import DataLoader
from transformers.data.data_collator import DataCollatorWithPadding
from tqdm.notebook import tqdm
import torch 

results = []
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorWithPadding(tokenizer)

testdataloader = test_dataloader(test_name, 64)
with torch.no_grad():
    for step, batch in enumerate(tqdm(testdataloader)):
        batch = batch.to('cuda')
        output = model(batch['input_ids'], batch['attention_mask'])
        predicted_labels = torch.sigmoid(output)
        results.extend(predicted_labels.cpu().detach().numpy().tolist())

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-efeade2dc67e1dd3.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-67b183b76ee90bd1.arrow


  0%|          | 0/844 [00:00<?, ?it/s]

In [146]:
import pandas as pd
df = pd.read_csv(test_name)
df['predict'] = results
df['pred_flag']=[True if i>0.5 else False for i in df['predict']]

In [147]:
df.to_csv(save_path, index=False)

In [148]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
y_test=df['flag']
y_pred=df['pred_flag']
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))
print(accuracy_score(y_test, y_pred))

0.639635335965705
0.6056394770396241
0.7610640290182216
0.9004780610732286


In [149]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.98      0.91      0.95     51767
        True       0.23      0.61      0.33      2201

    accuracy                           0.90     53968
   macro avg       0.61      0.76      0.64     53968
weighted avg       0.95      0.90      0.92     53968



## Post processing

In [240]:
from utils import (
    convert_to_ancestor_graph, 
    maximum_absorbance, 
    dataframe_to_ancestor_graph, 
    evaluate_groups,
    maximum_likelihood
)

from tqdm import tqdm


In [289]:
save_path = 'results/ccs/finetune.csv' # 'results/wordnet/lora.csv'
actual_tree = 'data/acm_ccs_clean.csv' # 'data/bansal_wordnet_true_pairs.csv'
post_process = maximum_likelihood # maximum_likelihood maximum_absorbance

df_t = pd.read_csv(save_path)

In [290]:
df_t['predict'] = np.log(df_t['predict'])

In [291]:
df_t['predict']

0       -0.323099
1       -0.463524
2       -0.245057
3       -0.326255
4       -0.279465
           ...   
21795   -0.058393
21796   -0.059638
21797   -0.188928
21798   -0.062616
21799   -0.109479
Name: predict, Length: 21800, dtype: float64

In [292]:
forest=[]
for g in tqdm(list(set(df_t.group))):
    df = post_process(df_t, g)
    forest.append(df)

  0%|          | 0/15 [00:00<?, ?it/s]100%|██████████| 15/15 [00:00<00:00, 217.39it/s]


In [293]:
res_v2=pd.concat(forest, ignore_index=True)


In [294]:
res_v2

,parent,child,group,compare
0,Time series analysis,Bayesian nonparametric models,10002950_2,Time series analysisBayesian nonparametric mod...
1,Time series analysis,Quantile regression,10002950_2,Time series analysisQuantile regression10002950_2
2,Time series analysis,Simulated annealing,10002950_2,Time series analysisSimulated annealing10002950_2
3,Time series analysis,Markov-chain Monte Carlo convergence measures,10002950_2,Time series analysisMarkov-chain Monte Carlo c...
4,Time series analysis,Causal networks,10002950_2,Time series analysisCausal networks10002950_2
...,...,...,...,...
5060,Security services,Access control,10002978_3,Security servicesAccess control10002978_3
5061,Security services,Graphical / visual passwords,10002978_3,Security servicesGraphical / visual passwords1...
5062,Security services,Digital rights management,10002978_3,Security servicesDigital rights management1000...
5063,Authentication,Authorization,10002978_3,AuthenticationAuthorization10002978_3


In [295]:
res_v2['group'].value_counts()

10010147_3     2094
10002951_5     1346
10002950_2      442
10010583_10     361
10010147_5      241
10010405_4      223
10010583_2      182
10003120_3       82
10003456_2       55
10002978_9       15
10002978_3       11
10002950_5        7
10010583_8        4
10010583_7        2
Name: group, dtype: int64

In [296]:
df_actual=pd.read_csv(actual_tree)
df_actual=df_actual[df_actual['type'] == 'test']
df_actual['compare']=df_actual['parent']+df_actual['child']+df_actual['group'].astype(str)

In [297]:
df_actual.group

23      10002950_2
24      10002950_2
25      10002950_2
26      10002950_2
27      10002950_2
           ...    
1838    10003120_3
1839    10003120_3
1840    10003120_3
1841    10003120_3
1842    10003120_3
Name: group, Length: 442, dtype: object

In [298]:
df_actual = dataframe_to_ancestor_graph(df_actual)
df_actual['group'].value_counts()

100%|██████████| 15/15 [00:00<00:00, 937.29it/s]


10010147_3     186
10002951_5     145
10002950_2     110
10003456_2      98
10010147_5      55
10010583_10     51
10003120_3      45
10010405_4      35
10010583_7      33
10010583_2      25
10010583_8      15
10002950_5      15
10002978_3      12
10002978_9       5
10002950_3       3
Name: group, dtype: int64

In [299]:
df_actual[df_actual.group=='10010147_3']

,parent,child,group,compare
110,Artificial intelligence,Matching,10010147_3,Artificial intelligenceMatching10010147_3
111,Artificial intelligence,Probabilistic reasoning,10010147_3,Artificial intelligenceProbabilistic reasoning...
112,Artificial intelligence,Computer vision,10010147_3,Artificial intelligenceComputer vision10010147_3
113,Artificial intelligence,Object identification,10010147_3,Artificial intelligenceObject identification10...
114,Artificial intelligence,Biometrics,10010147_3,Artificial intelligenceBiometrics10010147_3
...,...,...,...,...
291,Control methods,Computational control theory,10010147_3,Control methodsComputational control theory100...
292,Computer vision representations,Shape representations,10010147_3,Computer vision representationsShape represent...
293,Computer vision representations,Appearance and texture representations,10010147_3,Computer vision representationsAppearance and ...
294,Computer vision representations,Hierarchical representations,10010147_3,Computer vision representationsHierarchical re...


In [300]:
res_v2[res_v2.group=='10010147_3']

,parent,child,group,compare
442,Computer vision tasks,Matching,10010147_3,Computer vision tasksMatching10010147_3
443,Computer vision tasks,Probabilistic reasoning,10010147_3,Computer vision tasksProbabilistic reasoning10...
444,Computer vision tasks,Artificial intelligence,10010147_3,Computer vision tasksArtificial intelligence10...
445,Computer vision tasks,Computer vision,10010147_3,Computer vision tasksComputer vision10010147_3
446,Computer vision tasks,Object identification,10010147_3,Computer vision tasksObject identification1001...
...,...,...,...,...
2531,Computational control theory,Abstraction and micro-operators,10010147_3,Computational control theoryAbstraction and mi...
2532,Computational control theory,Control methods,10010147_3,Computational control theoryControl methods100...
2533,Computational control theory,Robotic planning,10010147_3,Computational control theoryRobotic planning10...
2534,Object detection,Object recognition,10010147_3,Object detectionObject recognition10010147_3


In [301]:
recall, precision, f1 = evaluate_groups(df_actual, res_v2)

100%|██████████| 15/15 [00:00<00:00, 732.46it/s]


In [302]:
recall

0.5545976766455171

In [303]:
precision

0.2623150088461571

In [304]:
f1

0.2651459080063267

In [188]:
df_common = pd.merge(df_actual, res_v2, on=['compare'], how='inner')

In [189]:
len(df_common)/len(df_actual)

0.40734309104440586

In [190]:
len(df_common)/len(res_v2)

0.559836345039209

In [89]:
recall = []
precision = []
f1 = []
for group in tqdm(list(set(df_actual.group))):
    group_actual = df_actual[df_actual.group == group]
    group_pred = res_v2[res_v2.group == group]
    recall.append(len(group_actual.merge(group_pred, on='compare')) / len(group_actual))
    precision.append(len(group_actual.merge(group_pred, on='compare')) / len(group_pred))
    if precision[-1] + recall[-1] == 0:
        f1.append(0)
    else:
        f1.append(2 * (precision[-1] * recall[-1]) / (precision[-1] + recall[-1]))

  0%|          | 0/114 [00:00<?, ?it/s]100%|██████████| 114/114 [00:00<00:00, 475.12it/s]


In [82]:
import numpy as np
np.mean(recall)

0.5121387919282124

In [83]:
np.mean(precision)

0.6256533225972404

In [84]:
np.mean(f1)

0.550320877862728